# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [45]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
import time
import numpy as np
from PIL import Image
import json
from collections import OrderedDict

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [46]:
# data_dir = 'flowers'
data_dir = '~/.pytorch/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [47]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                         transforms.RandomResizedCrop(224),
                                         transforms.RandomHorizontalFlip(p = 0.6),
                                         transforms.RandomVerticalFlip(p = 0.4),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485, 0.456, 0.406],
                                                              [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [48]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
num_output_classes = len(cat_to_name)
num_output_classes

102

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [49]:
## helper

images, labels = next(iter(trainloader))
print(labels.shape)
print(images.shape)
print(images.dtype)
print(labels)


print( "trainloader", len(trainloader))
print( "testloader", len(testloader))
print( "validloader", len(validloader))


torch.Size([64])
torch.Size([64, 3, 224, 224])
torch.float32
tensor([ 52,  49,   6,  80,  59, 101,  70,  40,  90,  59,  92,  92,  55,  68,
         51,  82,  65,  93,  73,  89,  88,  16,  76,  28,  78,  78,  82,  26,
         46,  82,  49,  99,  41, 100,  84,   5,  75,  33,   1,  41,  45,  36,
         49,  96, 100,  22,  94,  97,  76,  51,   6,  97,  66,  56,  77,   3,
         64,  74,  76,  39,  90,  83,  61,  32])
trainloader 103
testloader 13
validloader 13


In [50]:
# TODO: Build and train your network

# setup to pick up GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# get pre-trained model
model = models.resnet50(pretrained=True)
#print(model)

# freeze parameters - to prevent gradients and backprop
for param in model.parameters():
    param.requires_grad = False

classifier = nn.Sequential(nn.Linear(2048, 500),
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(500, num_output_classes),
                           nn.LogSoftmax(dim=1)
                          )
model.fc = classifier

# Initialization
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

# move model to cuda if available
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
traintime = time.time()

epochs = 3
training_loss = 0

train_losses, valid_losses = [], []

for epoch in range(epochs):
    model.train()
    for images, labels in trainloader:
        
        # move the variables to GPU
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()        
        
        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        training_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0

        # turn of dropouts
        model.eval()

        with torch.no_grad():                

            for images, labels in validloader:

                # move the variables to GPU
                images, labels = images.to(device), labels.to(device)
                
                start = time.time()

                logps = model(images)
                loss = criterion(logps, labels)
                test_loss += loss.item()

                # Calculate accuracy
                ps = torch.exp(logps)
                top_prob, top_class = ps.topk(1, dim=1)
                targets = labels.view(*top_class.shape)
                isEqual = top_class == targets                                                  
                accuracy += torch.mean(isEqual.type(torch.FloatTensor))
            
            train_losses.append(training_loss/len(trainloader))
            valid_losses.append(test_loss/len(validloader))
            
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {training_loss/len(trainloader):.3f}.. "
                  f"validation loss: {test_loss/len(validloader):.3f}.. "
                  f"Validation accuracy: {accuracy/len(validloader):.3f}")
            print(f"Device = {device}; Time per batch: {(time.time() - start)/len(validloader):.3f} seconds")
            
            training_loss = 0
            
            # switch back to training
            model.train()
            
print(f" \nTotal Time : {(time.time() - traintime)/60:.3f} minutes")

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.legend(frameon=False)

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [51]:
# TODO: Do validation on the test set
def test_network(model, testloader, device):
    start = time.time()
    test_loss = 0
    accuracy = 0

    # turn of dropouts
    model.eval()

    with torch.no_grad():                

        for images, labels in testloader:

            # move the variables to GPU
            images, labels = images.to(device), labels.to(device)

            logps = model(images)
            loss = criterion(logps, labels)
            test_loss += loss.item()

            # Calculate accuracy
            ps = torch.exp(logps)
            top_prob, top_class = ps.topk(1, dim=1)
            targets = labels.view(*top_class.shape)
            isEqual = top_class == targets                                                  
            accuracy += torch.mean(isEqual.type(torch.FloatTensor))

        print(f"Test loss: {test_loss/len(testloader):.3f}.. "
              f"Test accuracy: {accuracy/len(testloader):.3f}")
        print(f" Time per batch: {(time.time() - start):.3f} seconds")
        test_loss = 0

        # switch back to training
        model.train()

In [52]:

test_network(model, testloader, device)

Test loss: 4.632.. Test accuracy: 0.006
 Time per batch: 138.720 seconds


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
checkpoint = {'pretrained_model' : models.resnet50(pretrained=True),
              'input_size' : 2048,
              'output_size' : 102,
              'hidden_layer' : 500,
              'model_state_dict': model.state_dict(),
              'optim_state_dict' : optimizer.state_dict(),
              'class_to_idx': train_data.class_to_idx}

torch.save(checkpoint, 'trained_model_chpt.pth')

#train_data.class_to_idx

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_Checkpoint(filename):
    
    checkpoint = torch.load(filename)
    
    # get pre-trained model
    #model = checkpoint['pretrained_model']
    model = models.resnet50(pretrained=True)

    #model.to("cpu")
    
    # freeze parameters - to prevent gradients and backprop
    for param in model.parameters():
        param.requires_grad = False

    # TODO - this function should load for any architecture
    
    classifier = nn.Sequential(nn.Linear(2048, 500),
                               nn.ReLU(),
                               nn.Dropout(p=0.2),
                               nn.Linear(500, num_output_classes),
                               nn.LogSoftmax(dim=1)
                              )
    model.fc = classifier   
    
    
    
    
    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    model.class_to_idx = checkpoint['class_to_idx']
    
    return model

In [ ]:
# test saving and loading
new_model = load_Checkpoint('trained_model_chpt.pth')
new_model

In [ ]:
# adiitional test - check if loaded checkpoint performs as expected
device = "cpu"
new_model.to(device)
test_network(new_model, testloader, device)
    

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # TODO: Process a PIL image for use in a PyTorch model
    
    crop_size = 224
    new_width = 256
    # ImageNet mean and std    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    # 1. Resize with shortest side 256 and maintain aspect ratio
    width, height = image.size
    orig_ar = (width / height)    
    new_height = (new_width / orig_ar)
    image.thumbnail((new_width, new_height), Image.ANTIALIAS)
    w1, h1 = image.size

    # 2. center crop
    left = int((new_width - crop_size) / 2)
    top = int((new_height - crop_size) / 2)
    right = int((new_width + crop_size) / 2)
    bottom = int((new_height + crop_size) / 2)
    image = image.crop((left, top, right, bottom)) 
    w2, h2 = image.size
    
    print("original dim  : ", width, height)
    print("After thumbnail : ", w1, h1)
    print("center_crop : ", left, top, right, bottom)    
    print("After crop : ", w2, h2)
    #image.show()
    
    
    # 3. PIL to nparray - 0-255
    np_image = np.array(image)
    print("\n numpy image : ", np_image.min(), np_image.max(), np_image.dtype)
    # scale : 0 to 1 range
    np_image = np_image / 255.0
    print("numpy image : ", np_image.min(), np_image.max(), np_image.dtype)

    # 4. Normalize the image
    print(" image shape : ", np_image.shape)
    np_image = (np_image - mean) / std
    #print("normalized numpy image : ", np_image.mean(), np_image.std(), np_image.dtype)

    # 5. re-order color channels
    out_image = np.transpose(np_image, (2, 0, 1))
    print(" image shape : ", out_image.shape)
    
    return out_image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    print('*** inside imshow***')
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
# test  - process_image()
from PIL import Image

image_path = r'C:\Users\janan\.pytorch\flower_data\test\1' + '\image_06743.jpg'
image_path = r"C:\Learning\Nanodegree\pytorch\image_06743.jpg"
image = Image.open(image_path)

image_ndarray = process_image(image)

image_torch = torch.from_numpy(image_ndarray)

imshow(image_torch)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    image = Image.open(image_path)

    image_ndarray = process_image(image)

    # create a torch tensor of type float32
    image_torch = torch.from_numpy(image_ndarray).float()
    print(image_torch.dtype)

    # reshape to incorporate batch size
    batch_t = torch.unsqueeze(image_torch, 0)
    print(batch_t.shape)

    logps = model(batch_t)
    ps = torch.exp(logps)
    top_prob, top_idx = ps.topk(topk,dim=1)
    
    # index to class mapping
    idx_to_class = {value : key for key, value in model.class_to_idx.items()}

    # convert torch to numpy
    top_idx = top_idx[0].numpy()
    top_class = [ idx_to_class[entry] for entry in top_idx]
    
    return top_prob, top_class

In [ ]:
def view_classify(img, top_class, topk, cat_to_name, label_class):

    class_names = [ cat_to_name[item] for item in top_class]
    print(class_names)
    
    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)    
    ax1 = imshow(image_torch, ax = ax1)
    ax1.axis('off')
    
    # ax1.set_title(cat_to_name[label_class]) # ---> fix this!!
    
    y_pos = np.arange(topk)
    ax2.barh(np.arange(topk), list(reversed(top_prob[0])))
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(topk))
    ax2.set_yticklabels(reversed(class_names), size='small');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()
    

In [ ]:

# test prediction
with torch.no_grad():   
    
    image_path = r"C:\Learning\Nanodegree\pytorch\image_06743.jpg"
    image_path = r'C:\Users\janan\.pytorch\flower_data\test\1' + '\image_06743.jpg'
    #image_path = r'C:\Users\janan\.pytorch\flower_data\test\2' + '\image_05125.jpg'
    #image_path = r'C:\Users\janan\.pytorch\flower_data\test\76' + '\image_02550.jpg'
    label_class = [2] # this should be target label
    
    model = new_model
    topk = 5

    top_prob, top_class = predict(image_path, model, topk=5)
    
    

    image = Image.open(image_path)
    image_ndarray = process_image(image)
    image_torch = torch.from_numpy(image_ndarray)
    #imshow(image_torch)

    print(top_prob)
    print(top_class)
    
    view_classify(image_torch, top_class, topk, cat_to_name, label_class)

    
    

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
cat_to_name

type(cat_to_name)

sorted_cat_to_name = OrderedDict(sorted(cat_to_name.items()))

for k, v in sorted_cat_to_name.items():
    print (k, ":", v)

    
topk= 5
axs = imshow(image_torch, ax = plt)
axs.axis('off')
axs.title(top_class[0])
axs.show()
fig, ax = plt.subplots()
y_pos = np.arange(topk)
plt.barh(y_pos, top_prob[0])
plt.yticks(y_pos, top_class)
plt.ylabel('Flower Type')
plt.xlabel('Class Probability')


# References: 

PyTorch for Beginners: Image Classification using Pre-trained models - june 2019

https://www.learnopencv.com/pytorch-for-beginners-image-classification-using-pre-trained-models/